In [102]:
import pybullet as p
import pybullet_data
import time
import math
import random
import csv
import xlsxwriter

In [92]:
p.disconnect()


In [73]:
def applied_speed(speed):
    return (speed+2.1536)/0.1454


def zumoVel_to_bullet(zumoVel):
    return (28.0/400.0)*zumoVel

In [105]:
header = ["Time", "Error"]

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('GA-test.xlsx')

#Sheet names
name = "PD"

sheets = {}



In [103]:
#file = open("PID.csv", "w", newline="")
#writer = csv.writer(file, delimiter=";")
#writer.writerow(header)

#start the simulator in either GUI or DIRECT mode
physicsclient = p.connect(p.GUI) #change to "p.GUI" to visualize the simulation

p.configureDebugVisualizer(p.COV_ENABLE_WIREFRAME, 0)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)

#Set path to pybullet data, which is used to load the standard plane
p.setAdditionalSearchPath(pybullet_data.getDataPath())

#Load the plane
plainID = p.loadURDF("plane.urdf")

#Load the simplified zumo
robot = p.loadURDF("SimpleZumo.urdf",
                    [0,0,0.01])

#Set gravity
p.setGravity(0,0,-1000)
p.setRealTimeSimulation(0)
p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-65, cameraTargetPosition=[0,0,0])

In [96]:
def calculate_fitness(error):
    return 1/(error**2 + 0.00000000001)

def select_next_gen(list_of_gains, list_of_fitness, total_fitness, advancing_num = 40):
    next_gen_gains = []
    probs = []
    cum_prob = [] 
    prob_sum = 0

    for i in range(len(list_of_fitness)):
        prob = list_of_fitness[i]/total_fitness
        probs.append(prob)

        prob_sum += prob
        cum_prob.append(prob_sum)
    
    print("Maximal cummulated prob:")
    print(max(cum_prob))
    print()

    #We need to save the idx of the best chromosome to make it proceed to next generation
    best_idx = probs.index(max(probs))
    next_gen_gains.append(list_of_gains[best_idx])
    
    for i in range(advancing_num-1):
        r = random.random()
        n = 0

        while True: #for n in range(len(cum_prob)):
            if r < cum_prob[n]:
                if list_of_gains[n] not in next_gen_gains:
                    next_gen_gains.append(list_of_gains[n])
                    break
                else:
                    r = random.random()
                    n = -1

            n += 1
                
    return next_gen_gains


def crossover_and_mutate(gains, cross_val = 0.8, num_cross = 60, num_mutation = 39):
    next_generation_gains = []
    
    #Append the best from previous generation
    next_generation_gains.append(gains[0])

    n = 0

    while True: #for i in range(num_cross/2):
        parent1 = gains[random.randint(1, len(gains)-1)]
        parent2 = gains[random.randint(1, len(gains)-1)]

        #Make sure that the mates are different
        while parent2 == parent1:
            parent2 = gains[random.randint(1, len(gains)-1)]

        child1 = [cross_val*parent1[0]+(1-cross_val)*parent2[0], cross_val*parent1[1]+(1-cross_val)*parent2[1]]
        child2 = [cross_val*parent2[0]+(1-cross_val)*parent1[0], cross_val*parent2[1]+(1-cross_val)*parent1[1]]

        if child1 not in next_generation_gains:
            next_generation_gains.append(child1)
            next_generation_gains.append(child2)
            n += 1
        
        if n == num_cross/2:
            break

    for i in range(num_mutation):
        mutation = gains[random.randint(1, len(gains)-1)]

        mutation[0] = mutation[0] * random.uniform(0.75, 1.25)
        mutation[1] = mutation[1] * random.uniform(0.75, 1.25)

        next_generation_gains.append(mutation)

    return next_generation_gains
    

    


In [97]:
dist = 200

generations = 10
num_individuals = 100

gains = []
best_gains = []
fitness = []
fitness_sum = 0

gen_error = 0
gen_errors = []

for i in range(num_individuals):
    gains.append([random.uniform(0, 100), random.uniform(0, 100)])

for gen in range(generations): 
    for n in range(num_individuals):
        speedRight = 30.0
        speedLeft = 30.0

        VelRight = zumoVel_to_bullet(applied_speed(speedRight))
        VelLeft = zumoVel_to_bullet(applied_speed(speedLeft))

        kc = gains[n][0]
        kd = gains[n][1]

        lastError = 0
        error_total = 0      

        angle = p.getQuaternionFromEuler([0, 0, 0.15])
        p.resetBasePositionAndOrientation(robot, [0,0,0.01], angle) #[0,0,0,1])
        p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-75, cameraTargetPosition=[0,0,0])

        for i in range(100):
            p.stepSimulation()

        p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
        p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
        p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight)
        p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight)

        i = 0

        while True:
            p.resetDebugVisualizerCamera(cameraDistance=0.8, cameraYaw=0, cameraPitch=-75, cameraTargetPosition=[0,i/(450+(400-speedRight)),0])
            
            #i%4 svarer til at den opdaterer med samme interval som zumoen
            #hvert step (p.stepSimulation) svarer til 1/240 s. 4*1/240 = 16ms, hvilket svarer til zumoens korrektionsinterval
            if(i % 4 == 0):
                pos, orient = p.getBasePositionAndOrientation(robot)
                angle = p.getEulerFromQuaternion(orient)
                error = 0 - angle[2] * 180.0/math.pi

                error_total += abs(error)
                gen_error += error_total
                
                speedDiff = float(kc*error + kd*(error-lastError))
                
                lastError = error

                VelRight -=  zumoVel_to_bullet(speedDiff)
                VelLeft += zumoVel_to_bullet(speedDiff)

                VelRight = zumoVel_to_bullet(70) if VelRight < zumoVel_to_bullet(70) else (zumoVel_to_bullet(400) if VelRight > zumoVel_to_bullet(400) else VelRight)
                VelLeft = zumoVel_to_bullet(70) if VelLeft < zumoVel_to_bullet(70) else (zumoVel_to_bullet(400) if VelLeft > zumoVel_to_bullet(400) else VelLeft)

                p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
                p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft)
                p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight)
                p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight)

            p.stepSimulation()
            pos, orient = p.getBasePositionAndOrientation(robot)
            

            if pos[1] >= (dist/100):
                fit = calculate_fitness(error_total)
                fitness.append(fit)
                fitness_sum += fit

                break
            else:
                i += 1

    tmp_gains = select_next_gen(gains, fitness, fitness_sum, 60)
    gains = crossover_and_mutate(tmp_gains)
    best_gains.append(gains[0])
    gen_errors.append(gen_error/num_individuals)
    gen_error = 0

    print(f"Generation {gen}")
    print(f"Best gain: {gains[0]}  Error: {gen_errors[gen]}\n")

    fitness_sum = 0
    fitness = []

        

Maximal cummulated prob:
0.9999999999999999

Generation 0
Best gain: [18.98249796540693, 49.12039519113641]  Error: 27835.702722207556

Maximal cummulated prob:
1.0000000000000002

Generation 1
Best gain: [91.96210746347617, 65.67175770213184]  Error: 18074.008239501083

Maximal cummulated prob:
1.0

Generation 2
Best gain: [39.04312285574718, 73.5842589776413]  Error: 17352.479197709534

Maximal cummulated prob:
1.0000000000000007

Generation 3
Best gain: [72.25628641828784, 45.954511770746]  Error: 17644.226959925905

Maximal cummulated prob:
1.0000000000000002

Generation 4
Best gain: [35.021703967460816, 61.02390620988092]  Error: 16632.03459878091

Maximal cummulated prob:
1.0

Generation 5
Best gain: [31.895470481718785, 75.73290951632269]  Error: 16451.130840831054

Maximal cummulated prob:
0.9999999999999999

Generation 6
Best gain: [50.99977075104101, 58.33569469618466]  Error: 16631.96889031707

Maximal cummulated prob:
1.0000000000000002

Generation 7
Best gain: [48.76946288

In [99]:
sheets[name + str(iterations+1)] = workbook.add_worksheet(str(name) + str(iterations+1))
row = 0
graph_error = True

for i in range(iterations):
    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    #chart = workbook.add_chart({'type': 'line'})

    chart.add_series({
        'name': '',
        'categories': f'={name}{i}!$A$2:$A$500',
        'values':     f'={name}{i}!$B$2:$B$500',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': f'={name}{i}!$D$1',
                        'name_font': {'size': 14}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Time [s]', 'major_unit': 0.5}) 

    # Add y-axis label
    chart.set_y_axis({'name': 'Angular Error [degrees]'})

    chart.set_legend({'none': True})

    # Insert the chart into the worksheet.
    sheets[name + str(i)].insert_chart('D5', chart)

    #You can not insert the chart two times and it therefore necessary to creat the chart again
    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    #chart = workbook.add_chart({'type': 'line'})

    chart.add_series({
        'name': '',
        'categories': f'={name}{i}!$A$2:$A$500',
        'values':     f'={name}{i}!$B$2:$B$500',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': f'={name}{i}!$D$1',
                        'name_font': {'size': 12}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Time [s]', 'major_unit': 1}) 

    # Add y-axis label
    chart.set_y_axis({ 'major_unit': 0.1,
                        'min': -0.4, 'max': 0.4}) #'name': 'Angular Error [degrees]'

    chart.set_legend({'none': True})

    if i%2 == 0: 
        sheets[name + str(iterations+1)].insert_chart(f'A{row*11+1}', chart, {'x_scale': 0.59, 'y_scale': 0.75,})  
    else:
        sheets[name + str(iterations+1)].insert_chart(f'E{row*11+1}', chart, {'x_offset': 30, 'x_scale': 0.6, 'y_scale': 0.75,})
        row+=1

row = 0

#Make an chart over the generations' average error
if graph_error:
    sheets[name + str(iterations+2)] = workbook.add_worksheet(str(name) + str(iterations+2))
    for i in range(len(gen_errors)):
        sheets[name + str(iterations+2)].write(row, 0, i)
        sheets[name + str(iterations+2)].write(row, 1, gen_errors[i])
        row += 1


    chart = workbook.add_chart({'type': 'scatter',
                             'subtype': 'straight'})
    chart.add_series({
        'name': '',
        'categories': f'={name}{iterations+2}!$A$1:$A${len(gen_errors)}',
        'values':     f'={name}{iterations+2}!$B$1:$B${len(gen_errors)}',
        'data_labels': {'series_name': False}
    })

    # Add a chart title 
    chart.set_title({ 'name': 'Avg. Generation Error',
                        'name_font': {'size': 14}})

    # Add x-axis label
    chart.set_x_axis({'name': 'Generation', 'major_unit': 1}) 

    # Add y-axis label
    chart.set_y_axis({'name': 'Avg. error'})

    chart.set_legend({'none': True})

    # Insert the chart into the worksheet.
    sheets[name + str(iterations+2)].insert_chart('D5', chart)

In [109]:
workbook.close()

In [108]:
p.disconnect()


In [68]:
for i in range(700):
    p.stepSimulation()
    time.sleep(1/240)

In [5]:
pos, orient = p.getBasePositionAndOrientation(robot)

angle = p.getEulerFromQuaternion(orient)
print(pos)
print(angle[2])

(-0.0005336275790961077, -0.001762585590958836, -0.017168511378816243, 0.9998509142620599)
(0.07356550324080376, 3.9093242991709003, 0.020318889279408853)
-0.03433698491979333


In [7]:
VelRight = -40
VelLeft = -40

kc = 10
kd = 2

lastError = 0

for i in range(100):
    #p.stepSimulation()
    time.sleep(1/240)

p.setJointMotorControl2(robot, 0, p.VELOCITY_CONTROL, targetVelocity = VelLeft, maxVelocity = 100)
p.setJointMotorControl2(robot, 1, p.VELOCITY_CONTROL, targetVelocity = VelLeft, maxVelocity = 100)
p.setJointMotorControl2(robot, 2, p.VELOCITY_CONTROL, targetVelocity = VelRight, maxVelocity = 100)
p.setJointMotorControl2(robot, 3, p.VELOCITY_CONTROL, targetVelocity = VelRight, maxVelocity = 100)

In [191]:
#lad os få lidt ekstra information omkring joint 7
p.getJointInfo(robot, 2)


(2,
 b'Rev3',
 0,
 9,
 8,
 1,
 0.0,
 0.0,
 0.0,
 -1.0,
 0.0,
 0.0,
 b'Wheel1',
 (-1.0, 0.0, 0.0),
 (-0.05, 0.023999999999999994, 0.0),
 (0.0, 0.0, 0.0, 1.0),
 -1)